<a href="https://colab.research.google.com/github/tangcc35/DATA1030-Project-Cangcheng/blob/master/Sharpe_Style_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title User Inputs

# benchmark_id = 'AMCPX'
# benchmark_id = 'FBALX'
# benchmark_id = "FIVFX"
# portfolio_ids = ['^IRX','VGIT','VGLT','VCLT','VMBS','IWD','IWF','^RUT','BNDX','EFA','EEM']
# portfolio_ids = ['^GSPC', '^RUT', 'EFA', 'GBF','COMT']

# benchmark_id = "FIVFX"
# portfolio_ids = ['^IRX','VGIT','TLT','VCLT','VMBS','IWD','IWF','^RUT','EFA','EEM']
# date_range = ['2015-01', '2019-12']

# benchmark_id = "FIVFX"
# portfolio_ids = ['^IRX','^RUI','EFA','EEM']
# date_range = ['2010-01', '2019-12']

benchmark_id = "ACWI"
portfolio_ids = ['^IRX','^RUI','EFA','EEM']
date_range = ['2010-01', '2019-12']


rolling_window = 36

### **Selected Indexes**




##### **US Equities**
S&P500 (^GSPC)

Russell 1000 (IWF, ^RUI)  
Russell 1000 Value ('IWD', ^RLV)  
Russell 1000 Growth (IWF,^RLG)  

    Russell Top 200 (IWL)  
    Russell Top 200 Value (IWX)  
    Russell Top 200 Growth (IWY)

    Russell MidCap (IWP)   
    Russell MidCap Value (IWS)  
    Russell MidCap Growth (IWR)

Russell 2000 (^RUT, IWM)  
Russell 2000 Value (^RUJ, IWN)      
Russell 2000 Growth IWO (^RUO, IWO)  

##### **Intl Equities**
MSCI EAFE (EFA)   
MSCI Emerging Markets (EEM)

iShares MSCI ACWI ETF (ACWI)
iShares MSCI ACWI ex U.S. ETF (ACWX)

##### **US Fixed Income**
Treasury Bills 90-day (^IRX)    
Vanguard Intermediate-Term Treasury Index Fund ETF (VGIT)  
Vanguard Long-Term Treasury Index Fund ETF (VGLT)  
iShares 20+ Year Treasury Bond ETF (TLT)

iShares Government/Credit Bond ETF (GBF)

Vanguard Long-Term Corporate Index Fund ETF (VCLT)  
Vanguard Mortgage-Backed Securities Index Fund ETF (VMBS) 

##### **High Yield**
SPDR Bloomberg Barclays High Yield Bond ETF (JNK)  
iShares iBoxx $ High Yield Corporate Bond ETF (HYG)  
Vanguard High-Yield Corporate Fund Investor Shares (VWEHX)  

##### **Intl Fixed Income**
Vanguard Total International Bond Index Fund ETF (BNDX)  

##### **Real Estate**
Vanguard Real Estate Index Fund ETF Shares (VNQ)  
Fidelity MSCI Real Estate Index ETF (FREL)
     
Vanguard Global ex-U.S. Real Estate Index Fund ETF Shares (VNQI)

### **Code**

In [0]:
#@title Initialize / Import Modules
# import some modules
from pprint import pprint
import os
import pandas as pd
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
import plotly.graph_objects as go
import collections
import numpy as np
from pprint import pprint
from scipy.optimize import minimize

In [0]:
#@title Create Data
def create_data(ticker, start_date, end_date, frequency=None):
  old_start_date_string = start_date
  old_end_date_string = end_date
  decrement_start = 2
  decrement_end = 1
  year_start = int(old_start_date_string[:-3])
  month_start = int(old_start_date_string[-2:])
  year_end = int(old_end_date_string[:-3])
  month_end = int(old_end_date_string[-2:])
  month_start = month_start - decrement_start
  month_end = month_end + decrement_end
  if month_start < 1:
    month_start = 12
    year_start = year_start - 1

  if month_end > 12:
    month_end = 1
    year_end = year_end + 1

  new_start_date = str(year_start) + '-{:02d}'.format(month_start)
  new_end_date = str(year_end) + '-{:02d}'.format(month_end) 

  # Creates vector containing all dates in period
  dates = pd.date_range(new_start_date, new_end_date)
  dates = dates.to_frame()
  
  # Pulls historical price data for given ticker
  item_data = web.DataReader(ticker, 'yahoo', new_start_date, new_end_date)


  # Merges price data with full set of dates to account for missing dates.
  updated_data = dates.merge(right=item_data['Adj Close'], how='left', left_index=True, right_index=True)
  updated_data = updated_data.fillna(method='ffill')
  
  if(frequency == 'monthly'):
    month_ends = pd.date_range(new_start_date, new_end_date, freq='M')
    month_ends = month_ends.to_frame()
    updated_data = month_ends.merge(updated_data['Adj Close'], left_index=True, right_index=True)
   
  return(updated_data)


def create_dataset(benchmark, portfolios, date_range):
  start_date = date_range[0]
  end_date = date_range[1]

  bench_data = create_data(benchmark, start_date, end_date, 'monthly')['Adj Close']

  t_bill = create_data('^IRX', start_date, end_date, 'monthly')['Adj Close']
  dataset = pd.DataFrame({'^IRX':t_bill, benchmark:bench_data})
  dataset["%s_return" % (benchmark)] = dataset[benchmark].pct_change()
  dataset["^IRX_return"] = dataset['^IRX'].pct_change()

  cols = []
  for p in portfolios:
    cols.append("%s_return" % (p))
    portfolio_data = create_data(p, start_date, end_date, 'monthly')['Adj Close']
    dataset = pd.merge(dataset, portfolio_data, how='left', left_index=True, right_index=True)
    dataset.rename(columns={'Adj Close': p}, inplace=True)
  
    dataset["%s_return" % (p)] = dataset[p].pct_change()
  
  missing_cols = dataset.drop(dataset.index[0:1]).columns[dataset.drop(dataset.index[0:1]).isna().any()].tolist()
  if (len(missing_cols) != 0):
    return print("Columns are missing data ", missing_cols)
   
  return(dataset["%s_return" % (benchmark)].drop(dataset.index[0:1]), dataset[cols].drop(dataset.index[0:1]), dataset['^IRX_return'].drop(dataset.index[0:1]))

In [0]:
#@title Optimization
def opt_portfolio(X, y, wt_init=None, budget_constraint_flag=True, long_only_flag=True ):

  # internal function. will have "access" to X and y
  def min_func(weights):
    y_hat = np.dot(X, weights)
    SSResid = np.sum((y - y_hat) ** 2)
    return SSResid

  # counts
  nobs = X.shape[0]
  numx = X.shape[1]
  # initial weights
  if wt_init is None:
    wt_init = [1.0 / numx] * numx
  # check for budget constraint
  if budget_constraint_flag:
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
  else:
    cons = ()
  # check for long only constraint
  if long_only_flag:
    bnds = tuple((0, 1) for x in range(len(portfolio_ids)))
  else:
    bnds = None
  opt = minimize(min_func
          ,wt_init
          ,method='SLSQP'
          ,bounds=bnds
          ,constraints=cons
          ,tol=1e-10
        )
  opt_weights = opt['x']
  return opt_weights


In [0]:
#@title Rolling Optimization
def rolling_opt(X, y, window, wt_init=None, budget_constraint_flag=True, long_only_flag=True ):
  import copy
  if (window > len(X)):
    return print("Cannot compute; window too long; it is longer than entire time period.")
  else:
    windows = []
    weights = []
    for i in range(len(X) - window):
      i_beg = i
      i_end = i + window
      X_window = copy.deepcopy(X[i_beg:i_end])
      y_window = copy.deepcopy(y[i_beg:i_end])
      w = opt_portfolio(X_window, y_window, wt_init=wt_init, budget_constraint_flag=budget_constraint_flag, long_only_flag=long_only_flag)
      weights.append(w)
      windows.append([i_beg, i_end])
  weights = np.array(weights)
  return windows, weights

In [0]:
#@title Calc Results
def calc_results(X, y, wt):
  results = collections.OrderedDict()

  nobs = len(y) # number of observations
  numx = len(wt) # number of independent variables

  fund_ret = y
  fund_ret_hat = X @ wt   # dot product of wts and returns

  # active returns and vol
  act_ret_mean = (fund_ret - fund_ret_hat).mean()
  act_vol = (fund_ret - fund_ret_hat).std(ddof=1)
  info_ratio = act_ret_mean / act_vol

  # one sample t-test w null hypothesis mean is zero
  act_ret_nullvalue = 0.0
  tstat = (act_ret_mean - act_ret_nullvalue) / (act_vol / np.sqrt(nobs))

  # rsquared
  r_sqrd = 1 - ((fund_ret - fund_ret_hat).var() / fund_ret.var())
  # adjusted rsquared
  r_sqrd_adj = 1.0 - ( ( (1.0 - r_sqrd)*(nobs-1) ) / (nobs - numx - 1) )

  # now populate result dictionary
  results['act_ret_mean']=act_ret_mean
  results['act_vol']=act_vol
  results['info_ratio']=info_ratio
  results['tstat']=tstat
  results['r_sqrd'] = r_sqrd
  results['r_sqrd_adj'] = r_sqrd_adj

  #results['ids'] = portfolio_ids
  results['wt'] = wt

  return results

In [0]:
#@title Generate Table 
def generate_table(benchmark_id, portfolio_ids, date_range, X, y):

  analysis_option_sets = [
    {'label': "no_con", 'budget_constraint': False, 'long_only': False}
    ,{'label': 'budget', 'budget_constraint': True, 'long_only': False}
    ,{'label': 'budget&longonly', 'budget_constraint': True, 'long_only': True}
  ]

  summarydata = []

  for options in analysis_option_sets:
    wt = opt_portfolio(X, y, wt_init=None, budget_constraint_flag=options['budget_constraint'], long_only_flag=options['long_only'])
    results = calc_results(X, y, wt)
    results['label'] = options['label']
    summarydata.append(results)

#  pprint(summarydata)

  table_data = []

  row_labels = ['MthlyAvgRet','MthlySdev','InfoRatio','T-stat','Rsqrd','RsqrdAdj']

  wt_label = []
  for widx, w in enumerate(wt):
    wt_label.append('{} wt%'.format(portfolio_ids[widx]))
  row_labels = row_labels + wt_label + ['TotWt']

  #print(row_labels)

  table_data.append(row_labels)
  for idx, results in enumerate(summarydata):
    #print('>>>>', idx)
    column_data = []
    column_data.append(results['act_ret_mean']*100)
    column_data.append(results['act_vol']*100)
    column_data.append(results['info_ratio'])
    column_data.append(results['tstat'])
    column_data.append(results['r_sqrd'])
    column_data.append(results['r_sqrd_adj'])
    for widx, w in enumerate(results['wt']):
      column_data.append(results['wt'][widx]*100)
    column_data.append(np.sum(results['wt'])*100)
    table_data.append(column_data)

  numrows = len(table_data[0])  # can look at first column to get number of rows
  row_height = 30
  figure_height = (numrows+6)*(row_height*1.25)

  headers = ["", "No Constraints", "Budget constraint(sum wts = 1)", "Budget and Long-only constraint"]
  cfmt = ['0.2f']
  table_trace = go.Table(
                header=dict(values=headers, align="center", fill = dict(color='#C2D4FF'), font = dict(color = 'black', size = 14))
                ,cells=dict(values=table_data
                           #,format=['12s']
                           #, format=[['']*10,['0.2f']*10,['0.2f']*10,['0.2f']*10] #, fill = dict(color='#f2f3f4'), font = dict(size = 14)
                           #, format=[[''],['0.2f'],['0.2f'],['0.2f']] #, fill = dict(color='#f2f3f4'), font = dict(size = 14)
                           , format=['','0.2f','0.2f','0.2f'], height=40 #, fill = dict(color='#f2f3f4')
                           #, font = dict(size = 14, ['rgb(255,0,0)' if type(v)=='str' else 'rgb(10,10,10)' for v in vals])
#                    ,height=30
                
                    
                )
  )
  layout = go.Layout(
    autosize=False,
    width=1500,
    height=figure_height
    ,margin=dict(l=20, r=20, t=40, b=60),
    paper_bgcolor="LightSteelBlue"
  )
  
  fig = go.Figure(data=[table_trace], layout=layout)
    
  fig.show()


  return
  

In [0]:
#@title Rolling weight plot
def generate_rolling_weight_plot(rolling_periods, rolling_weights):
  
  df = pd.DataFrame(index=X.index[rolling_window:], data=rolling_weights)

  fig = go.Figure()
  for i, portfolio in enumerate(portfolio_ids):
    fig.add_trace(go.Scatter(x=df.index, y=df[i], 
                      mode='none' # override default markers+lines
                      ,stackgroup='one' # define stack group
                      , name=portfolio))

  fig.update_layout(go.Layout(
      autosize=False,
      width=1500,
      height=500,
      title = 'Portfolio Weights Over Time Rolling Period (%s months)' % (rolling_window),
      xaxis_tickformat = '%B <br>%Y',
      #xaxis_title="Time",
      yaxis_title="Weight",
      yaxis_range=(0, 1)

      ,margin=dict(l=20, r=20, t=60, b=20)
      ,paper_bgcolor="LightSteelBlue"
      ,legend_orientation="h"

    )
  )



  fig.show()


### **Run Analysis**

In [0]:
#@title Retrieve data / create data set
y, X, rf = create_dataset(benchmark_id, portfolio_ids, date_range)

In [0]:
#@title Run table
generate_table(benchmark_id, portfolio_ids, date_range, X, y)

In [0]:
#@title Run rolling weights
rolling_periods, rolling_weights = rolling_opt(X, y, rolling_window, wt_init=None, budget_constraint_flag=True, long_only_flag=True )
generate_rolling_weight_plot(rolling_periods, rolling_weights)